In [ ]:
import numpy as np
from PIL import Image, ImageFilter
import time
import numpy as np

In [ ]:
eps = 0.1
lr = 0.001
alpha = 0.1

In [ ]:
'''
#just some little tests

a = time.time()
img = Image.open("imgs/IMG_0002.PNG").convert("RGBA")
b = time.time()
print("load and convert:", b - a)
a = time.time()
img2 = Image.open("imgs/IMG_0042.jpg").convert("RGBA").resize((275,192)).rotate(-2, expand=1)
b = time.time()
print("load, convert, resize, rotate:", b - a)

a = time.time()
img.paste(img2, (245,45), mask=img2)
b = time.time()
print("paste:", b - a)

a = time.time()
img = img.filter(ImageFilter.GaussianBlur(5))
b = time.time()
print("blur:", b - a)

a = time.time()
img = img.resize((275,192), resample=Image.Resampling.LANCZOS)
b = time.time()
print("lanczos:", b - a)

img.show()
'''

In [ ]:
def mse(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.mean((a - b) ** 2)

def stamper(goal, stamp, trans):
    stamp = stamp.rotate(trans[2], expand=1)
    goal.paste(stamp, (int(trans[0]), int(trans[1])), mask=stamp)
    return goal

def optimizer(goal, stamp):
    goalSize = goal.size
    bestTrans = np.zeros(3) #x, y, rot
    for i in range(goal.size[0], -1, -1):
        gradient = np.ones(3) * 100
        momentum = np.zeros(3)
        while (np.linalg.norm(gradient) > 0.1):
            gt = goal.filter(ImageFilter.GaussianBlur(i))
            current = stamper(goal, stamp, bestTrans).filter(ImageFilter.GaussianBlur(i))
            fdx = stamper(goal, stamp, bestTrans + np.array([i * eps, 0, 0])).filter(ImageFilter.GaussianBlur(i))
            fdy = stamper(goal, stamp, bestTrans + np.array([0, i * eps, 0])).filter(ImageFilter.GaussianBlur(i))
            fdr = stamper(goal, stamp, bestTrans + np.array([0, 0, i * eps])).filter(ImageFilter.GaussianBlur(i))
            
            ecurrent = mse(gt, current)
            ex = mse(gt, fdx) # punish positions outside of the image
            ey = mse(gt, fdy)
            er = mse(gt, fdy)
            
            gradient[0] = (ecurrent - ex) / eps
            gradient[1] = (ecurrent - ey) / eps
            gradient[2] = (ecurrent - er) / eps
            
            momentum = alpha * momentum - lr * gradient
            bestTrans = bestTrans - lr * gradient + alpha * momentum
            
            print(i, ecurrent, gradient, bestTrans)
            
            #oh god, what now? How to do the scales?
    return bestTrans

In [ ]:
a = Image.open("a.PNG").convert("RGBA")
b = Image.open("a.PNG").convert("RGBA").rotate(180)
optimizer(a, b)